In [67]:
!pip install wikipedia

In [80]:
topic = "RadixDLT"

In [69]:
from langchain_openai import AzureChatOpenAI

deployment_name = "leitura_documento_gpt4o"
openai_api_version="2023-05-15"
model = "gpt-4o"
temperature = 0.9

llm = AzureChatOpenAI(azure_deployment=deployment_name, openai_api_version=openai_api_version, temperature=temperature)

In [70]:
from langchain_community.retrievers import WikipediaRetriever

wiki_retriever = WikipediaRetriever(load_max_docs=3)

docs = wiki_retriever.invoke(topic)
docs

[Document(page_content='Bitcoin (abbreviation: BTC; sign: ₿) is the first decentralized cryptocurrency. Nodes in the peer-to-peer bitcoin network verify transactions through cryptography and record them in a public distributed ledger, called a blockchain, without central oversight. Consensus between nodes is achieved using a computationally intensive process based on proof of work, called mining, that guarantees the security of the bitcoin blockchain. Mining consumes large quantities of electricity and has been criticized for its environmental impact.\nBased on a free market ideology, bitcoin was invented in 2008 by Satoshi Nakamoto, an unknown person. Use of bitcoin as a currency began in 2009, with the release of its open-source implementation.:\u200ach. 1\u200a In 2021, El Salvador adopted it as legal tender. Bitcoin is currently used more as a store of value and less as a medium of exchange or unit of account. It is mostly seen as an investment and has been described by many schola

In [71]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import RunnablePassthrough

prompt = ChatPromptTemplate.from_template(
    """
    Answer the question based only on the context provided.
    Context: {context}
    Question: {question}
    """
)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


chain_wiki = (
    {"context": wiki_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [72]:
chain_wiki.invoke(
    "Who created BTC and when?"
)

'Bitcoin (BTC) was invented in 2008 by Satoshi Nakamoto, an unknown person.'

In [73]:
# template for the title
title_template = PromptTemplate(
    input_variables = ['topic'], 
    template='write me a tweet about {topic}'
)

# template for the tweet
tweet_template = PromptTemplate(
    input_variables = ['title', 'wikipedia_research'], 
    template='write me a tweet on this title TITLE: {title} while leveraging this wikipedia reserch:{wikipedia_research} '
)

In [76]:
title_chain = (
    title_template
    | llm
    | StrOutputParser()
)

title = title_chain.invoke(topic)
title

'🚀 Bitcoin is more than just digital currency; it’s a revolution in finance! 🌍💡 Empowering individuals, decentralizing economies, and offering financial freedom. 💸✨ #Bitcoin #Crypto #Blockchain'

In [81]:
tweet_chain = (
    {"title": title_chain, "wikipedia_research": wiki_retriever | format_docs}
    | tweet_template
    | llm
    | StrOutputParser()
)

tweet = tweet_chain.invoke(topic)
tweet

'🚀 Exciting times ahead with #RadixDLT! 🌐 Innovating #DeFi with unparalleled scalability, security, and seamless decentralized finance solutions. Say goodbye to blockchain bottlenecks and hello to limitless possibilities, powered by cutting-edge tech like reversible cellular automata! 🔗 #Crypto #Blockchain #Fintech #Innovation 🚀'